In [23]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# data = pd.read_csv("./data/_PHEME_text.csv")
data = pd.read_csv("./data/_PHEME_text_twtToken.csv")
target = pd.read_csv("./data/_PHEME_target.csv")

In [8]:
data.head()

,text,Event,token
0,BREAKING: Armed man takes hostage in kosher gr...,charliehebdo,"['breaking', 'armed', 'man', 'takes', 'hostage..."
1,"#CharlieHebdo killers dead, confirmed by genda...",charliehebdo,"['#', 'killers', 'dead', 'confirmed', 'gendarm..."
2,"Top French cartoonists Charb, Cabu, Wolinski, ...",charliehebdo,"['top', 'french', 'cartoonists', 'charb', 'cab..."
3,Police have surrounded the area where the #Cha...,charliehebdo,"['police', 'surrounded', 'area', '#', 'attack'..."
4,PHOTO: Armed gunmen face police officers near ...,charliehebdo,"['photo', 'armed', 'gunmen', 'face', 'police',..."


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
from csv import reader,writer
import operator as op
import string
from sklearn import neighbors

### Reader - tfidf - knn

In [11]:
# #Read data from corpus
r = reader(open("./data/_PHEME_text_twtToken.csv",'r'))
abstract_list = []
score_list = []
institute_list = []
row_count = 0
for row in list(r)[1:]:
    institute,score,abstract = row[0], row[1], row[2]
    if len(abstract.split()) > 0:
      institute_list.append(institute)
      score = float(score)
      score_list.append(score)
      abstract = abstract.translate(string.punctuation).lower()
      abstract_list.append(abstract)
      row_count = row_count + 1

# print("Total processed data: ", row_count)

# #Vectorize (TF-IDF, ngrams 1-4, no stop words) using sklearn -->
# vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,4),
#                      min_df = 0, stop_words = 'english', sublinear_tf=True)
# response = vectorizer.fit_transform(abstract_list)
# classes = score_list
# feature_names = vectorizer.get_feature_names()

# clf = neighbors.KNeighborsRegressor(n_neighbors=1)
# clf.fit(response, classes)
# clf.predict(response)

ValueError: could not convert string to float: 'charliehebdo'

## TFIDF

In [ ]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,
                        ngram_range=(1, 2), 
                        stop_words='english')

In [7]:
text = list(data['text'])
rumor = list(target['target'])

vectorizer = TfidfVectorizer(
            min_df=0.0,
            analyzer="char",
            sublinear_tf=True,
            ngram_range=(1,3),
            max_features=5000
        )

X = vectorizer.fit_transform(text)

## TFIDF from BERT_Scratch

In [9]:
import nltk
# Uncomment to download "stopwords"
nltk.download("stopwords")
from nltk.corpus import stopwords

def text_preprocessing(s):
    """
    - Lowercase the sentence
    - Change "'t" to "not"
    - Remove "@name"
    - Isolate and remove punctuations except "?"
    - Remove other special characters
    - Remove stop words except "not" and "can"
    - Remove trailing whitespace
    """
    s = s.lower()
    # Change 't to 'not'
    s = re.sub(r"\'t", " not", s)
    # Remove @name
    s = re.sub(r'(@.*?)[\s]', ' ', s)
    # Isolate and remove punctuations except '?'
    s = re.sub(r'([\'\"\.\(\)\!\?\\\/\,])', r' \1 ', s)
    s = re.sub(r'[^\w\s\?]', ' ', s)
    # Remove some special characters
    s = re.sub(r'([\;\:\|•«\n])', ' ', s)
    # Remove stopwords except 'not' and 'can'
    s = " ".join([word for word in s.split()
                  if word not in stopwords.words('english')
                  or word in ['not', 'can']])
    # Remove trailing whitespace
    s = re.sub(r'\s+', ' ', s).strip()
    
    return s

[nltk_data] Downloading package stopwords to /Users/june/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
data = pd.read_csv("./data/_PHEME_text_twtToken.csv")
target = pd.read_csv("./data/_PHEME_target.csv")

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val =\
    train_test_split(data, target, test_size=0.1, random_state=2020)

In [17]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer

# Preprocess text
tokens = list(data['token'])
rumor = list(target['target'])
X_train_preprocessed = X_train['token']
X_val_preprocessed = X_val['token']
# X_train_preprocessed = np.array([text_preprocessing(text) for text in X_train])
# X_val_preprocessed = np.array([text_preprocessing(text) for text in X_val])

# Calculate TF-IDF
tf_idf = TfidfVectorizer(ngram_range=(1, 3),
                         binary=True,
                         smooth_idf=False)
X_train_tfidf = tf_idf.fit_transform(X_train_preprocessed)
X_val_tfidf = tf_idf.transform(X_val_preprocessed)

CPU times: user 438 ms, sys: 73 ms, total: 511 ms
Wall time: 623 ms


In [19]:
print(X_train['token'].shape)
print(X_train_tfidf.shape) # 훈련 데이터를 tfidf score로 표현한 데이터

(5221,)
(5221, 64758)


In [24]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

def get_auc_CV(model):
    """
    Return the average AUC score from cross-validation.
    """
    # Set KFold to shuffle data before the split
    kf = StratifiedKFold(5, shuffle=True, random_state=1)

    # Get AUC scores
    auc = cross_val_score(
        model, X_train_tfidf, y_train, scoring="roc_auc", cv=kf)

    return auc.mean()

In [ ]:
from sklearn.naive_bayes import MultinomialNB

res = pd.Series([get_auc_CV(MultinomialNB(i))
                 for i in np.arange(1, 10, 0.1)],
                index=np.arange(1, 10, 0.1))

best_alpha = np.round(res.idxmax(), 2)
print('Best alpha: ', best_alpha)

plt.plot(res)
plt.title('AUC vs. Alpha')
plt.xlabel('Alpha')
plt.ylabel('AUC')
plt.show()

In [ ]:
# Compute predicted probabilities

# X_data = pd.concat([X_train_tfidf, X_val_tfidf], axis=0)
# y_data = pd.concat([X_train_tfidf, X_val_tfidf], axis=0)

X_test = test_data.text.values
y_test = test_data.isRumor.values

X_preprocessed = np.array([text_preprocessing(text) for text in X])
X_tfidf = tf_idf.transform(X_preprocessed)
X_test_preprocessed = np.array([text_preprocessing(text) for text in X_test])
X_test_tfidf = tf_idf.transform(X_test_preprocessed)

nb_model = MultinomialNB(alpha=1.8)
nb_model.fit(X_tfidf, y)
probs = nb_model.predict_proba(X_test_tfidf)

# Evaluate the classifier
evaluate_roc(probs, test_data.isRumor)